## Анализ базы данных приложения для чтения книг

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
query = '''
SELECT * FROM books LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` Содержит данные о книгах: - `book_id` — идентификатор книги; - `author_id` — идентификатор автора; - `title` — название книги; - `num_pages` — количество страниц; - `publication_date` — дата публикации книги; - `publisher_id` — идентификатор издателя.

In [3]:
query = '''
SELECT * FROM authors LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` Содержит данные об авторах: - `author_id` — идентификатор автора; - `author` — имя автора.

In [4]:
query = '''
SELECT * FROM publishers LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` Содержит данные об издательствах: - `publisher_id` — идентификатор издательства; - `publisher` — название издательства;

In [5]:
query = '''
SELECT * FROM ratings LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` Содержит данные о пользовательских оценках книг: - `rating_id` — идентификатор оценки; - `book_id` — идентификатор книги; - `username` — имя пользователя, оставившего оценку; - `rating` — оценка книги.

In [6]:
query = '''
SELECT * FROM reviews LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` Содержит данные о пользовательских обзорах на книги: - `review_id` — идентификатор обзора; - `book_id` — идентификатор книги; - `username` — имя пользователя, написавшего обзор; - `text` — текст обзора.


1. Посчитайте количество книг, выпущенных после 1 января 2000 года

In [7]:
# Для этого посчитаем все строки в таблице books в которых publication_date больше '2000-01-01'
query = '''
SELECT 
    COUNT(*)
FROM
    books
WHERE 
    publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


2. Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги

In [27]:

query = '''
SELECT 
    books.title AS title,
    AVG(ratings.rating) AS avg_rating,
    COUNT(distinct reviews.review_id) AS cnt_review_id
FROM 
    books
LEFT OUTER JOIN ratings ON ratings.book_id = books.book_id
LEFT OUTER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY 
    books.book_id
ORDER BY 
    COUNT(reviews.review_id) DESC;
'''
pd.io.sql.read_sql(query, con = engine)

,title,avg_rating,cnt_review_id
0,Twilight (Twilight #1),3.662500,7
1,The Hobbit or There and Back Again,4.125000,6
2,The Catcher in the Rye,3.825581,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
...,...,...,...
995,Essential Tales and Poems,4.000000,0
996,Leonardo's Notebooks,4.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
998,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0


3. Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры)

In [9]:
query = '''
SELECT 
    publishers.publisher AS publisher,
    COUNT(books.book_id) AS cnt_books
FROM -- к таблице books добавим табилцу publishers по полю publisher_id
    books
LEFT OUTER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE 
    num_pages > '50'
GROUP BY
    publisher
ORDER BY 
    cnt_books DESC
LIMIT 1; 
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,cnt_books
0,Penguin Books,42


4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками

In [10]:
query = '''
SELECT
    author 
FROM
    authors 
WHERE 
    author_id =
        (SELECT 
            author_id
        FROM 
            (SELECT 
                books.author_id,
                AVG(ratings.rating) AS avg_rating
            FROM 
                ratings
            LEFT JOIN books ON books.book_id = ratings.book_id 
            WHERE 
                ratings.book_id IN
                    (SELECT 
                         book_id
                    FROM 
                         ratings
                    GROUP BY 
                        book_id
                    HAVING 
                        COUNT(ratings.rating_id) >= '50')
            GROUP BY 
                author_id
            ORDER BY 
                avg_rating DESC
            LIMIT 1) AS Sub);
                    

'''
pd.io.sql.read_sql(query, con = engine)

,author
0,J.K. Rowling/Mary GrandPré


5. Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [11]:
query = '''
SELECT 
    AVG(Sub.cnt_reviews) AS avg_reviews
FROM
    (SELECT  
        reviews.username,
        COUNT(reviews.review_id) AS cnt_reviews
    FROM 
        reviews
    WHERE 
        username IN
            (SELECT 
                ratings.username
            FROM 
                ratings
            GROUP BY
                ratings.username
            HAVING 
                COUNT(ratings.rating_id) > '50')
    GROUP BY
        reviews.username) AS Sub;
'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333
